### Callbacks / Event-Handlers
 
- In Python sind Funktionen wie man sagt **first class citizens**. Funktionen sind Objekte wie alle anderen und k&ouml;nnen u.a. Variabeln zugewiesen werden oder (anderen) Funktionen als Argumente &uuml;bergeben werden.  

- Eine Funktion, welche einer anderen Funktion als Argument &uuml;bergeben wird, wird **Callback** genannt.
Oft wird diese Funktion beim Eintreffen eines sog. **Events** 
(Tastendruck, Mausklick, Mausklick auf Button, Ausw&auml;hlen einer Option in einem Menu,...) aufgerufen und dann auch **Event-Handler** genannt.  

- Das Sicherstellen, dass der Callback 
zu einem sp&auml;teren Zeitpunkt z.B. von einem Button-Objekt aufgerufen wird, nennt sich auch **Registrierung des Callbacks**.

***
**Ein erstes Beispiel**
***

In [ ]:
# wird spaeter als Callback verwendet
def square(x):
    return int(x)**2

In [ ]:
f = square
f.__name__

In [ ]:
# Funktionsnamen und Typnamen ausgeben
# f.__name__ ist der Name der Funktion

f = square
print(f,  int)
print(f.__name__, int.__name__)

In [ ]:
def str2list(s, f):
    '''trenne den String s beim ',' -> Liste
       wende f auf Listenelemente an
    '''   
    lst = s.split(',')
    return [f(x) for x in lst]

In [ ]:
# int('1') verhaelt sich wie eine Funktion, 
# gibt einen Integer zurueck und kann als 
# Callback verwendet werden

str2list('1,2,3,4,5', int)

In [ ]:
# Funktion square als Callbacks verwenden
str2list('1,2,3,4,5', square)

### Managen von Callbacks
Wir wollen Elemente einer Liste bearbeiten.
Je nach Typ des Listenelements soll etwas mit dem Element gemacht werden.  

Wir benutzen einen Dictionary `callbacks`. 
F&uuml;r einen Typ (z.b. `int`) kann 
`callbacks[int]` dann eine **Liste** von Funktionen enthalten, welche
der Reihe nach zum Modifizieren der Listenelemente dieses Typs verwendet werden, z.B.


```python
callbacks[int] = [int, double]
```

Die Funktion   
`register_callback(typename, fun, remove = False)`  
erlaubt das Hinzuf&uuml;gen und Entfernen von Callbacks.   

**Die Registrierung von Callbacks bei Widgets in Jupyterlab folgt im Wesentlichen diesem Muster.**

In [ ]:
# Dictionary zum Speichern der Callbacks
callbacks = {}

# Funktion zum Verwalten der Callbacks
def register_callback(typename, fun, remove = False):
    '''registriere fun als Callback fuer den Typ typename
       
       Ist remove = True, wird der Callback entfernt 
    '''
    if remove:
        if fun in callbacks[typename]:
            callbacks[typename].remove(fun)
            print('unregistered callback {} for type {}'\
                  .format(fun.__name__, typename.__name__)
                 )
        
    elif fun in callbacks.get(typename, []):
        print('Callback {} already registered for type {}'\
              .format(fun.__name__, typename.__name__)
             )
        
    else:
        callbacks.setdefault(typename,[]).append(fun)
        print('Callback {} registered for type {}'\
              .format(fun.__name__,typename.__name__))

In [ ]:
# Funktion zum Modifizieren von Listenelementen mittels 
#  Callbacks aus dem Dictionary callbacks

def format_data(data):
    res = []
    for x in data:
        fun_list = callbacks.get(type(x), [])
        for f in fun_list:
            x = f(x)
        res.append(x)
    return res

In [ ]:
data = [1, 2.298, 'foo']

In [ ]:
# noch sind keine Callbacks registriert
# die Liste kommt unveraendert zurueck

format_data([1, 2.23, 'foo'])

***
**Beachte:**  
Wird nachfolgende Zelle ein zweites Mal ausgef&uuml;hrt, so
werden neue Funktionen `double` und `simplify` erzeugt.  
In der n&auml;chsten Zelle werden dann diese Funktionen
als Callbacks registriert, ohne die alten Funktionen zu &uuml;berschreiben.  

Auf die alten Funktionen kann nun nicht mehr via die Variabelnamen 
double und simplify zugegriffen werden.
Das **verunm&ouml;glicht das Entfernen** der alten Funktionen als Callbacks mit unserer Funktion `register_callback`.
***

In [ ]:
# callbacks
def double(x):
    return 2*x
def simplify(x):
    return round(x, 1)

In [ ]:
# callbacks registrieren
# Integer verdoppeln und dann in String umwandeln
# floats auf eine Kommastelle runden
# Bei Strings Methode capitalize anwenden 
# (ersten Buchstaben gross machen, Rest klein)

register_callback(int, double)
register_callback(int, str)
register_callback(float, simplify)
register_callback(str, str.capitalize)

In [ ]:
# Funktionsnamen im Dictionary callbacks anzeigen
{k:[f.__name__ for f in v] for k,v in callbacks.items()}

In [ ]:
# Listenelemente modifizieren

format_data([1, 2.23, 'foo'])